In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm() 

Gen RAM Free: 10.7 GB  | Proc size: 15.8 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
import os
import time

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K
from PIL import Image
from tensorflow.keras import Model, Sequential
from tensorflow.keras import models, optimizers
from tensorflow.keras.layers import Reshape, Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda, Dropout
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from pathlib import Path

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tqdm import tqdm

In [0]:
class Recognizer:

    def __init__(self):
        tf.logging.set_verbosity(tf.logging.ERROR)

        self.__IMG_DIMENSIONS = 128

        self.__model: Model = None

    def build_model(self, dr: 0.3, learning_rate=0.0001):
        """

        :param dr:
        :param learning_rate:
        :return:
        """

        input_shape = ((self.__IMG_DIMENSIONS ** 2) * 3,)
        convolution_shape = (self.__IMG_DIMENSIONS, self.__IMG_DIMENSIONS, 3)
        
                
        seq_conv_model = Sequential()
        
        seq_conv_model.add(Reshape(input_shape=input_shape, target_shape=convolution_shape))
        seq_conv_model.add(Conv2D(32, kernel_size=(4, 4), activation='relu'))
        seq_conv_model.add(Conv2D(32, kernel_size=(4, 4), activation='relu'))
        seq_conv_model.add(MaxPooling2D(pool_size=(2, 2)))
        seq_conv_model.add(Dropout(dr))
        seq_conv_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        seq_conv_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
        seq_conv_model.add(MaxPooling2D(pool_size=(2, 2)))
        seq_conv_model.add(Dropout(dr))
        seq_conv_model.add(Flatten())
        seq_conv_model.add(Dense(128, activation='relu'))
        seq_conv_model.add(Dropout(0.5))
        seq_conv_model.add(Dense(64, activation='sigmoid'))

        print(seq_conv_model.summary())

        input_x1 = Input(shape=input_shape)
        input_x2 = Input(shape=input_shape)

        output_x1 = seq_conv_model(input_x1)
        output_x2 = seq_conv_model(input_x2)

        distance_l1 = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))([output_x1, output_x2])

        outputs = Dense(1, activation='sigmoid')(distance_l1)

        self.__model = Model([input_x1, input_x2], outputs)

        self.__model.compile(loss='binary_crossentropy',
                             optimizer=optimizers.Adam(lr=learning_rate),
                             metrics=['binary_accuracy'])

        self.__model.summary()

        return self.__model

    def prepare_images_from_dir(self, dir_path, flatten=True):
        """

        :param self:
        :param dir_path:
        :param flatten:
        :return:
        """
        images = list()
        images_names = os.listdir(dir_path)

        for imageName in images_names:

            image = Image.open(dir_path + imageName)
            resize_image = image.resize((self.__IMG_DIMENSIONS, self.__IMG_DIMENSIONS))
            array = list()
            for x in range(self.__IMG_DIMENSIONS):
                sub_array = list()
                for y in range(self.__IMG_DIMENSIONS):
                    sub_array.append(resize_image.load()[x, y])
                array.append(sub_array)
            image_data = np.array(array)
            image = np.array(np.reshape(image_data, (self.__IMG_DIMENSIONS, self.__IMG_DIMENSIONS, 3)))
            images.append(image)

        if flatten:
            images = np.array(images)
            return images.reshape((images.shape[0], self.__IMG_DIMENSIONS ** 2 * 3)).astype(np.float32)
        else:
            return np.array(images)

    def fit(self, X, Y, hyper_parameters):

        initial_time = time.time()
        self.__model.fit(X, Y,
                         batch_size=hyper_parameters['batch_size'],
                         epochs=hyper_parameters['epochs'],
                         callbacks=hyper_parameters['callbacks'],
                         validation_split=0.2,
                         verbose=1
                         )
        final_time = time.time()
        eta = (final_time - initial_time)

        time_unit = 'seconds'
        if eta >= 60:
            eta = eta / 60
            time_unit = 'minutes'

        print('Elapsed time acquired for {} epoch(s) -> {} {}'.format(hyper_parameters['epochs'], eta, time_unit))

    def evaluate(self, test_X, test_Y):
        return self.__model.evaluate(test_X, test_Y)

    def predict(self, X):
        predictions = self.__model.predict(X)
        return predictions

    def summary(self):
        self.__model.summary()

    def save_model(self, file_path):
        self.__model.save(file_path)

    def load_model(self, file_path):
        self.__model = models.load_model(file_path)

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
def load_images_and_targets(img_dir, img_dimen=128):
    """

    :param img_dir:
    :param output_dir:
    :param img_dimen:
    :return:
    """
    sub_dir_list = os.listdir(img_dir)
    images = list()
    targets = list()

    
    for idx in tqdm(range(len(sub_dir_list))):

        label = idx
        image_names = os.listdir(dir_path.joinpath(sub_dir_list[idx]))
        for image_path in image_names:
            path = dir_path.joinpath(sub_dir_list[idx], image_path)
            try:

                img = img_to_array(load_img(path, target_size=(img_dimen, img_dimen))) / 255.
                images.append(img)
                targets.append(label)
            except Exception as e:
                print('WARNING : File {} could not be processed.'.format(path))
                print(e)

    images = np.array(images)

    left_samples = list()
    right_samples = list()
    labels = list()
    
    print(images.shape)

    for i in tqdm(range(len(targets))):
        for j in range(len(targets)):
            left_samples.append(images[i])
            right_samples.append(images[j])

            left_target = targets[i]
            right_target = targets[j]
            # Yi = 1 ; if both images contain the same person
            # Yi = 0; if both images contain different people
            if left_target == right_target:
                labels.append(1)
            else:
                labels.append(0)

    x_left = np.array(left_samples)
    x_right = np.array(right_samples)
    y = np.array(labels)

    print('X left shape: {}'.format(x_left.shape))
    print('X right shape: {}'.format(x_right.shape))
    print('Y shape: {}'.format(y.shape))

    return x_left, x_right, y

In [0]:
project_dir = Path('drive/My Drive/BDCA Assignment')
models_dir = project_dir.joinpath('models')

IMG_DIMEN = 128
dir_path = project_dir.joinpath('images_trial')
out_path = project_dir.joinpath('output_images')
testing_dir = project_dir.joinpath('custom_images')

In [0]:
dir_path.exists()

True

In [0]:
out_path.exists()

True

In [0]:
models_dir.exists()

True

In [45]:
testing_dir.exists()

True

In [0]:
X1, X2, Y = load_images_and_targets(img_dir=dir_path,
                                    img_dimen=IMG_DIMEN)

100%|██████████| 198/198 [00:00<00:00, 6874.60it/s]


(198, 128, 128, 3)
X left shape: (39204, 128, 128, 3)
X right shape: (39204, 128, 128, 3)
Y shape: (39204,)


In [0]:
np.save('{}/x1.npy'.format(out_path), X1)
np.save('{}/x2.npy'.format(out_path), X2)
np.save('{}/y.npy'.format(out_path), Y)

In [0]:
X1 = np.load('{}/x1.npy'.format(out_path))
X2 = np.load('{}/x2.npy'.format(out_path))
Y = np.load('{}/y.npy'.format(out_path))

X1 = X1.reshape((X1.shape[0], IMG_DIMEN ** 2 * 3)).astype(np.float32)
X2 = X2.reshape((X2.shape[0], IMG_DIMEN ** 2 * 3)).astype(np.float32)

print(X1.shape)
print(X2.shape)
print(Y.shape)

(39204, 49152)
(39204, 49152)
(39204,)


In [0]:
recognizer = Recognizer()
# recognizer.load_model('models/model.h5')

In [0]:
callbacks = [
    EarlyStopping(monitor='val_loss',
                  patience=2,
                  verbose=1)

]

parameters = {
    'batch_size': 256,
    'epochs': 50,
    'callbacks': callbacks
}

In [0]:
recognizer.build_model(dr=0.4)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 128, 128, 3)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 125, 125, 32)      1568      
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 122, 122, 32)      16416     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 61, 61, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 61, 61, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 59, 59, 64)        18496     
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 57, 57, 64)       

In [0]:
recognizer.fit(X=[X1, X2],
               Y=Y,
               hyper_parameters=parameters)

Train on 31363 samples, validate on 7841 samples
Epoch 1/50
31363/31363 [==============================] - 156s 5ms/sample - loss: 0.1304 - binary_accuracy: 0.9744 - val_loss: 0.3301 - val_binary_accuracy: 0.9796
Epoch 2/50
31363/31363 [==============================] - 150s 5ms/sample - loss: 0.1286 - binary_accuracy: 0.9744 - val_loss: 0.3172 - val_binary_accuracy: 0.9796
Epoch 3/50
31363/31363 [==============================] - 151s 5ms/sample - loss: 0.1273 - binary_accuracy: 0.9744 - val_loss: 0.3207 - val_binary_accuracy: 0.9796
Epoch 4/50
31363/31363 [==============================] - 150s 5ms/sample - loss: 0.1264 - binary_accuracy: 0.9744 - val_loss: 0.2996 - val_binary_accuracy: 0.9796
Epoch 5/50
31363/31363 [==============================] - 150s 5ms/sample - loss: 0.1248 - binary_accuracy: 0.9744 - val_loss: 0.2495 - val_binary_accuracy: 0.9796
Epoch 6/50
31363/31363 [==============================] - 151s 5ms/sample - loss: 0.1240 - binary_accuracy: 0.9744 - val_loss: 0.17

In [0]:
scores = list ()
labels = list ()
for image in custom_images:
    label = list ()
    score = list ()
    for sample in class_1_images:
        image, sample = image.reshape((1, -1)), sample.reshape((1, -1))
        score.append(recognizer.predict([image, sample])[0])
        label.append(0)
    for sample in class_2_images:
        image, sample = image.reshape((1, -1)), sample.reshape((1, -1))
        score.append(recognizer.predict([image, sample])[0])
        label.append(1)
    labels.append(label)
    scores.append(score)

scores = np.array(scores)
labels = np.array(labels)

for i in range(custom_images.shape[0]):
    index = np.argmax(scores[i])
    label_ = labels[i][index]
    print('IMAGE {} is {} with confidence of {}'.format(i + 1, label_, scores[i][index][0]))